In [1]:
import torch
import torch.nn as nn
import os
import torchvision.models as models
from DataLoader import prepare_data_loaders
import copy
torch.manual_seed(42)

[INFO] loading the training and validation dataset...
[INFO] training dataset contains 19998 samples...
[INFO] validation dataset contains 5000 samples...


In [2]:
def print_model_size(mdl):
    torch.save(mdl.state_dict(), "tmp.pt")
    print("%.2f MB" %(os.path.getsize("tmp.pt")/1e6))
    os.remove('tmp.pt')

In [3]:
data_loader, data_loader_test = prepare_data_loaders()

In [4]:
def train_epoch(model, criterion, optimizer, data_loader, device,epoch):
    model.train()
    
    epoch_loss = 0.0
    num_batches = len(data_loader)
    
    for batch_idx, (image, target) in enumerate(data_loader):
        image, target = image.to(device), target.to(device)
        
        # Forward pass
        output = model(image)
        
        # Calculate loss
        loss = criterion(output, target)
        
        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # Accumulate batch loss
        epoch_loss += loss.item()
        
        # Print batch loss (optional)
        # print(f"Batch [{batch_idx + 1}/{num_batches}], Loss: {loss.item():.4f}")
    
    # Calculate average epoch loss
    avg_epoch_loss = epoch_loss / num_batches
    print(f"Epoch = {epoch+1} || Training Loss: {avg_epoch_loss:.4f}")
    

In [5]:
def evaluate(model, criterion, data_loader, device,epoch):
    
    model.eval()
    
    epoch_loss = 0.0
    
    correct_predictions = 0
    total_predictions = 0
    
    num_batches = len(data_loader)
    
    with torch.no_grad():
       
        for image, target in data_loader:
            image, target = image.to(device), target.to(device)
            output = model(image)
            loss = criterion(output, target)
            # Accumulate batch loss
            epoch_loss += loss.item()
            
            # Calculate accuracy
            _, predicted = torch.max(output, 1)  # Get the predicted class index
            correct_predictions += (predicted == target).sum().item()
            total_predictions += target.size(0)
            
    # Calculate average epoch loss
    avg_epoch_loss = epoch_loss / num_batches
    accuracy = correct_predictions / total_predictions
    
    print(f"Epoch = {epoch+1} || Test Loss: {avg_epoch_loss:.4f} || Test Accuracy: {accuracy:.4f}")
        

In [6]:
class MobileNet(torch.nn.Module):
    def __init__(self):
        super(MobileNet, self).__init__()
        self.model = models.mobilenet_v2(weights='MobileNet_V2_Weights.DEFAULT')  
        
        # for param in self.model.parameters():
        #     param.requires_grad = False
        
        self.model.classifier[1] = nn.Sequential(
            nn.Linear(in_features=self.model.classifier[1].in_features,out_features=512),
            nn.LeakyReLU(negative_slope=0.02,inplace=False),
            nn.BatchNorm1d(num_features=512),
            nn.Dropout(p=0.4,inplace=False),
            nn.Linear(in_features=512,out_features=2),
            nn.Softmax(dim=1))
        
        # print(self.model)

    def forward(self, x):
        x = self.model(x)
        return x

In [7]:
model = MobileNet()

In [8]:
print_model_size(model)

11.76 MB


In [9]:
import platform
chip = platform.processor()

if chip == 'arm':
    backend = 'qnnpack'
elif chip in ['x86_64', 'i386']:
    backend = 'fbgemm'
else:
    raise SystemError("Backend is not supported")

print(f"Using {backend} backend engine for {chip} CPU")

torch.backends.quantized.engine = backend

Using fbgemm backend engine for x86_64 CPU


In [10]:
criterion = nn.CrossEntropyLoss(reduction='mean')
optimizer = torch.optim.AdamW(model.parameters(), lr = 0.0001)

In [11]:

from torch.quantization.quantize_fx import prepare_fx, convert_fx,prepare_qat_fx

example_inputs = (torch.randn(1, 3, 28, 28),)
qconfig = {
    "": torch.quantization.get_default_qat_qconfig(backend),
    "module_name": [
    #    ("features.13", None),    
    #    ("features.14", None),
    #    ("features.15", None),
    #    ("features.16", None),
    #    ("features.17", None),
    ]
}
#model_prepared = prepare_fx(model.eval(), qconfig,example_inputs)
model_prepared = prepare_qat_fx(model.train(), qconfig, example_inputs)

/home/ss/miniconda3/lib/python3.12/site-packages/torch/ao/quantization/fx/prepare.py:1751: UserWarning: Passing a QConfig dictionary to prepare is deprecated and will not be supported in a future version. Please pass in a QConfigMapping instead.
  warnings.warn(
/home/ss/miniconda3/lib/python3.12/site-packages/torch/ao/quantization/observer.py:220: UserWarning: Please use quant_min and quant_max to specify the range for observers.                     reduce_range will be deprecated in a future release of PyTorch.
  warnings.warn(
/home/ss/miniconda3/lib/python3.12/site-packages/torch/ao/quantization/fx/utils.py:857: UserWarning: QConfig must specify a FixedQParamsObserver or a FixedQParamsFakeQuantize for fixed qparams ops, ignoring QConfig(activation=functools.partial(<class 'torch.ao.quantization.fake_quantize.FusedMovingAvgObsFakeQuantize'>, observer=<class 'torch.ao.quantization.observer.MovingAverageMinMaxObserver'>, quant_min=0, quant_max=255, reduce_range=True){'factory_kwargs':

In [12]:
for nepoch in range(20):
    train_epoch(model_prepared, criterion, optimizer, data_loader, torch.device('cpu'),nepoch)
    model_quantized = copy.deepcopy(model_prepared)
    model_quantized = convert_fx(model_quantized.eval())
    evaluate(model_quantized,criterion, data_loader_test,torch.device('cpu'),nepoch)
    

/home/ss/miniconda3/lib/python3.12/site-packages/PIL/TiffImagePlugin.py:890: UserWarning: Truncated File Read
  warnings.warn(str(msg))


Epoch = 1 || Training Loss: 0.6394
Epoch = 1 || Test Loss: 0.6031 || Test Accuracy: 0.6842
Epoch = 2 || Training Loss: 0.5876
Epoch = 2 || Test Loss: 0.5736 || Test Accuracy: 0.7200
Epoch = 3 || Training Loss: 0.5653
Epoch = 3 || Test Loss: 0.5580 || Test Accuracy: 0.7352
Epoch = 4 || Training Loss: 0.5530
Epoch = 4 || Test Loss: 0.5475 || Test Accuracy: 0.7518
Epoch = 5 || Training Loss: 0.5415
Epoch = 5 || Test Loss: 0.5451 || Test Accuracy: 0.7524
Epoch = 6 || Training Loss: 0.5347
Epoch = 6 || Test Loss: 0.5446 || Test Accuracy: 0.7512
Epoch = 7 || Training Loss: 0.5251
Epoch = 7 || Test Loss: 0.5282 || Test Accuracy: 0.7730
Epoch = 8 || Training Loss: 0.5178
Epoch = 8 || Test Loss: 0.5252 || Test Accuracy: 0.7756
Epoch = 9 || Training Loss: 0.5149
Epoch = 9 || Test Loss: 0.5240 || Test Accuracy: 0.7756
Epoch = 10 || Training Loss: 0.5096
Epoch = 10 || Test Loss: 0.5217 || Test Accuracy: 0.7814
Epoch = 11 || Training Loss: 0.5011
Epoch = 11 || Test Loss: 0.5188 || Test Accuracy: 0.

In [13]:
model_quantized = copy.deepcopy(model_prepared)
model_quantized = convert_fx(model_quantized.eval())
evaluate(model_quantized,criterion, data_loader_test,torch.device('cpu'),0)


Epoch = 1 || Test Loss: 0.5069 || Test Accuracy: 0.7930


In [14]:
print_model_size(model_quantized)

3.30 MB


In [15]:
traced_net = torch.jit.trace(model_quantized, torch.randn(1,3,28,28))

torch.jit.save(traced_net,'QATDogCatMobileNetV2.pt')